In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

In [ ]:
def wilcoxon_twosided_with_effects(base: np.ndarray, ours: np.ndarray, zero_method: str = 'wilcox'):
    """
    计算“双侧”Wilcoxon结果 + 效应量。
    差值 d = base - ours（>0 表示基线误差更大 → 你的方法更好）
    """
    # 去 NA & 形状检查
    mask = np.isfinite(base) & np.isfinite(ours)
    x = base[mask].astype(np.float64)
    y = ours[mask].astype(np.float64)
    if x.shape != y.shape:
        raise ValueError("基线与你的方法长度不一致")
    if x.size == 0:
        raise ValueError("有效样本为 0")

    d = x - y
    n_total = d.size
    n_zero  = int((d == 0).sum())
    n_eff   = int((d != 0).sum())
    n_pos   = int((d > 0).sum())
    n_neg   = int((d < 0).sum())
    if n_eff == 0:
        raise ValueError("全部差值为 0：Wilcoxon 无法计算")

    # 双侧 Wilcoxon（只报告这个 p 值）
    res_two = wilcoxon(x, y, zero_method=zero_method, alternative='two-sided', mode='auto')

    # 为了精确计算 rank-biserial r，获取 R+（不报告其 p 值）
    res_gt = wilcoxon(x, y, zero_method=zero_method, alternative='greater', mode='auto')
    R_plus = float(res_gt.statistic)
    S = n_eff * (n_eff + 1) / 2.0               # 非零差参与排名的秩总和
    r_rb = (2.0 * R_plus / S) - 1.0             # rank-biserial correlation（与Wilcoxon一致）
    cles = 0.5 * (r_rb + 1.0)                   # Common Language Effect Size

    # 配对 Cohen's d_z 与 Hedges' g
    d_mean = d.mean()
    d_std  = d.std(ddof=1)
    dz = np.nan if d_std == 0 else d_mean / d_std
    if n_eff > 2 and np.isfinite(dz):
        J = 1.0 - 3.0 / (4.0 * n_eff - 9.0)     # Hedges 小样本修正
        g = J * dz
    else:
        g = np.nan

    median_diff = float(np.median(d))

    # —— 输出（仅双侧）——
    print("=== Wilcoxon（双侧）与效应量 ===")
    print(f"有效配对数 n_eff      : {n_eff} / 总配对数 n_total: {n_total}（0差值: {n_zero}）")
    print(f"T 统计量（min秩和）    : {res_two.statistic:.6f}")
    print(f"p 值（two-sided）     : {res_two.pvalue:.3e}")
    print("--- 效应量 ---")
    print(f"rank-biserial r      : {r_rb:.3f}")
    print(f"CLES（P[base>ours]） : {cles:.3f}")
    print(f"Cohen's d_z（配对）  : {dz:.3f}")
    print(f"Hedges' g            : {g:.3f}")
    print(f"成对差 d 的中位数    : {median_diff:.6f}")

    return {
        "n_total": n_total,
        "n_zero": n_zero,
        "n_eff": n_eff,
        "wilcoxon_T": float(res_two.statistic),
        "wilcoxon_p_two_sided": float(res_two.pvalue),
        "rank_biserial_r": float(r_rb),
        "CLES": float(cles),
        "cohen_dz": float(dz),
        "hedges_g": float(g),
        "median_diff": float(median_diff),
    }

In [ ]:
# ========================
# 主流程（按需修改文件路径）
# ========================
ours_path =            # 你的方法误差文件地址
baseline_path =        # 基线误差文件地址

def main(ours_path: str, baseline_path: str, zero_method: str = 'wilcox'):
    ours = pd.read_csv(ours_path, header=None, dtype=np.float64).squeeze("columns").to_numpy()
    base = pd.read_csv(baseline_path, header=None, dtype=np.float64).squeeze("columns").to_numpy()
    if ours.shape != base.shape:
        raise ValueError("两个文件的长度不一致")
    _ = wilcoxon_twosided_with_effects(base, ours, zero_method=zero_method)

if __name__ == "__main__":
    main(ours_path, baseline_path, zero_method='wilcox')